In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from ib_insync import *
import datetime as dt
from IPython.display import display,clear_output
util.startLoop()
ib = IB()


In [2]:
ib.connect()

<IB connected to 127.0.0.1:7497 clientId=1>

In [ ]:
from ib_insync import *
import pandas as pd
import numpy as np
import datetime as dt
import os

ib = IB()
ib.connect()

# Strategy parameters
sma_s = 20  # SMA period for Bollinger Bands
freq = "1 min"
units = 1000
session_duration = dt.timedelta(hours=5)  # Run the session for 5 hours
session_start = dt.datetime.now()  # Start time of the session
end_time = session_start + session_duration  # Calculate the end time

cfd = CFD("EUR", currency="USD")
ib.qualifyContracts(cfd)
conID = cfd.conId
contract = Forex('EURUSD')
ib.qualifyContracts(contract)

def onBarUpdate(bars, hasNewBar):
    global df, last_bar
    if bars[-1].date > last_bar:
        last_bar = bars[-1].date
        df = pd.DataFrame(bars)[["date", "open", "high", "low", "close"]].iloc[:-1]
        df.set_index("date", inplace=True)
        ############### BOLLINGER BANDS STRATEGY #################
        df = df[["close"]].copy()
        df["SMA"] = df.close.rolling(sma_s).mean()  # Middle band
        df["std"] = df.close.rolling(sma_s).std()
        df["Upper"] = df["SMA"] + (df["std"] * 2)  # Upper band
        df["Lower"] = df["SMA"] - (df["std"] * 2)  # Lower band
        df.dropna(inplace=True)
        # Define position: 1 (long) if prices below the lower band,
        # -1 (short) if prices above the upper band, else neutral
        df["position"] = np.where(df["close"] < df["Lower"], 1, 0)
        df["position"] = np.where(df["close"] > df["Upper"], -1, df["position"])
        #########################################################
        # Trading
        target = df["position"][-1] * units
        execute_trade(target=target)
        os.system('cls')
        print(df)
    else:
        try:
            trade_reporting()
        except:
            pass

def execute_trade(target):
    global current_pos
    # 1. Get current position
    try:
        current_pos = [pos.position for pos in ib.positions() if pos.contract.conId == conID][0]
    except:
        current_pos = 0
    # 2. Identify required trades
    trades = target - current_pos

    # Trade execution
    if trades > 0:
        side = "BUY"
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)
    elif trades < 0:
        side = "SELL"
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)
    else:
        pass

def trade_reporting():
    global report

    fill_df = util.df([fs.execution for fs in ib.fills()])[["execId", "time", "side", "cumQty", "avgPrice"]].set_index("execId")
    profit_df = util.df([fs.commissionReport for fs in ib.fills()])[["execId", "realizedPNL"]].set_index("execId")
    report = pd.concat([fill_df, profit_df], axis=1).set_index("time").loc[session_start:]
    report = report.groupby("time").agg({"side": "first", "cumQty": "max", "avgPrice": "mean", "realizedPNL": "sum"})
    report["cumPNL"] = report.realizedPNL.cumsum()

    os.system('cls')
    print(df, report)

if __name__ == "__main__":
    bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr='1 D',
        barSizeSetting=freq,
        whatToShow='MIDPOINT',
        useRTH=True,
        formatDate=2,
        keepUpToDate=True
    )
    last_bar = bars[-1].date
    bars.updateEvent += onBarUpdate

    # Stop trading session after 5 hours
    while True:
        ib.sleep(5)  # Check every 5 seconds
        if dt.datetime.now() >= end_time:  # If the session duration has been met
            execute_trade(target=0)  # Close open position
            ib.cancelHistoricalData(bars)  # Stop stream
            ib.sleep(10)
            try:
                trade_reporting()  # Final reporting
            except:
                pass
            print("Session Stopped.")
            ib.disconnect()
            break
        else:
            pass


In [ ]:
ib.disconnect()

ib.disconnect()

In [14]:
print("he")

he
